Thinking like a software developer if you are an economist
===

Suppose I have a box of dice that contains a 4-sided die, a 6-sided die, an 8-sided die, a 12-sided die, and a 20-sided die, each of them with sides labeled from 1 on up sequentially. 
If you have ever played Dungeons & Dragons, you know what I am talking about. 
Suppose I select a die from the box at random, roll it, and truthfully report to you that I get a 6. 
What is the probability that I rolled each die?

Notes:

1. We want to solve this with Monte Carlo rather than just solving the math
2. We'd like a reasonable general solution, but not a big mess of abstractions
3. We'd like the code to be readable and ideally have the same shape as the problem space
4. And we'll use Python classes because OO is so cool these days

In [125]:
import random
import numpy as np

The objects in our world (dice) have little more than:

1. a name 
2. a way to roll themselves and return a result

We define the interface: a way to make a die, a way to get its name, and a way to get the result of a roll: Die(sides), die.name, die.draw()
Because we are software developers, we avoid adding arbitrary constraints, e.g.

1. The results will be integers (maybe the next problem will use picture dice)
2. The names will be strings (we don't care)
3. We even know the set of possible results (maybe the next problem will have continuous valued results)

In [126]:
class Die(object):
    def __init__(self, sides, name=None):
        self.sides = sides
        self.name = name if name is not None else sides
    def draw(self):
        return random.randint(1, self.sides)

Sanity Tests.
==

We love quick tests, even 3 lines of code will be wrong.

So, we:
1. Check a six-sided die gives sane results over a few rolls
2. Check our interface works for other simple objects (e.g. possibly biased coins,) if it's hard to write the coin class, we probably got the die class wrong...

In [127]:
d = Die(6)
print( [ d.draw() for i in range(10)] )
print()

class Coin(object):
    def __init__(self, bias=0):
        self.bias = bias
        if not bias:
            self.name = 'FairCoin'
        else:
            self.name = 'Coin:%s' % bias
    def draw(self):
        h = random.uniform(-1, 1) < self.bias
        return 'Heads' if h else 'Tails'
    
print([Coin().draw() for i in range(10)])
print([Coin(bias=-1).draw() for i in range(10)])
print([Coin(bias=1).draw() for i in range(10)])
print([Coin(bias=.75).draw() for i in range(10)])

[4, 3, 2, 6, 4, 3, 6, 2, 6, 5]

['Tails', 'Heads', 'Heads', 'Heads', 'Tails', 'Tails', 'Tails', 'Heads', 'Tails', 'Tails']
['Tails', 'Tails', 'Tails', 'Tails', 'Tails', 'Tails', 'Tails', 'Tails', 'Tails', 'Tails']
['Heads', 'Heads', 'Heads', 'Heads', 'Heads', 'Heads', 'Heads', 'Heads', 'Heads', 'Heads']
['Heads', 'Heads', 'Heads', 'Heads', 'Tails', 'Tails', 'Heads', 'Tails', 'Heads', 'Heads']


Trial is a class that handles making repeated draws and aggregating the results of those draws.
==

Note that:
1. It doesn't mention dice in its code: it just picks a source and assumes the source has a name and a method called draw().
2. It an abstract class: it doesn't even know how to pick a source.
3. When it prints a table summary, it converts its dictionary of dictionaries into a numpy array of counts via the utility method dictsToTbl(). dictsToTbl deserves to be a library function because it's
doing a generic, but non-trivial, thing that is not specific to our problem space.
4. Once we have our numpy array of counts, we can use numpy to easily convert to frequencies, scale to percentages, etc.

In [128]:
class Trial(object):
    def __init__(self):
        self.results = {} # { Result: { Source: count }}
        
    def _add(self, name, result):
        results = self.results
        if result not in results:
            results[result] = {}
        results[result][name] = results[result].get(name, 0) + 1
        
    def run(self, nDraws=1000000):
        for i in range(nDraws):
            src = self.chooseSource()
            self._add(src.name, src.draw())
            
    def printSummary(self):
        values, rows, cols = dictsToTbl(self.results)
        # It is left as an exercise to the reader to use their favorite pretty printing module to
        # draw a table with nice row and column headers...
        print('Cols are:', cols)
        print('Rows are:', rows)
        print((norm(values) * 10000).astype(int) / 100.)
        
def dictsToTbl(dd):
    rowNames = sorted(dd.keys())
    colNames = set()
    for r in dd.values():
        colNames.update(r.keys())
    colNames = sorted(colNames)
    ret = np.zeros((len(rowNames), len(colNames)))
    for rn, row in dd.items():
        ri = rowNames.index(rn)
        for cn, v in row.items():
            ci = colNames.index(cn)
            ret[ri,ci] = v
    return ret, rowNames, colNames
            
def norm(ar):
    return ar / np.sum(ar, axis=1, keepdims=True)


A UniformSelectionTrial is just a Trial that samples with replacement from a collection of source items.
==

Again, it doesn't know anything about dice.

So, we just give it the dice, run the trial, and print the results...

In [129]:
class UniformSelectionTrial(Trial):
    def __init__(self, sources):
        super(UniformSelectionTrial, self).__init__()
        self.sources = sources
        
    def chooseSource(self):
        return random.choice(self.sources)

dice = [ Die(n) for n in [ 4, 6, 8, 12, 20 ] ]
t = UniformSelectionTrial(dice)
t.run()

t.printSummary()

Cols are: [4, 6, 8, 12, 20]
Rows are: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[[  37.17   24.47   18.63   12.33    7.37]
 [  36.92   24.73   18.54   12.26    7.54]
 [  37.25   24.57   18.44   12.31    7.41]
 [  37.15   24.74   18.35   12.36    7.37]
 [   0.     39.11   29.45   19.54   11.88]
 [   0.     39.24   29.48   19.53   11.73]
 [   0.      0.     48.37   32.13   19.48]
 [   0.      0.     48.26   32.27   19.46]
 [   0.      0.      0.     62.91   37.08]
 [   0.      0.      0.     62.46   37.53]
 [   0.      0.      0.     62.09   37.9 ]
 [   0.      0.      0.     62.12   37.87]
 [   0.      0.      0.      0.    100.  ]
 [   0.      0.      0.      0.    100.  ]
 [   0.      0.      0.      0.    100.  ]
 [   0.      0.      0.      0.    100.  ]
 [   0.      0.      0.      0.    100.  ]
 [   0.      0.      0.      0.    100.  ]
 [   0.      0.      0.      0.    100.  ]
 [   0.      0.      0.      0.    100.  ]]


Sanity Check
==

We have a jar with 1000 coins. One is double headed. We pick a coin at random, flip it 10 times. If it came up heads 10 times, what's the chance we picked the two-headed coin?

In [130]:
class Counter(object):
    def __init__(self, source, value, nDraws):
        self.source = source
        self.value = value
        self.nDraws = nDraws
        self.name = self.source.name
    def draw(self):
        total = 0
        for i in range(self.nDraws):
            if self.source.draw() == self.value:
                total += 1
        return total
    
coins = [ Coin(bias=1) ] + [ Coin() for i in range(999) ]
counters = [ Counter(coin, 'Heads', 10 ) for coin in coins ]
t = UniformSelectionTrial(counters)
t.run()
t.printSummary()

Cols are: ['Coin:1', 'FairCoin']
Rows are: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[[   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [   0.    100.  ]
 [  51.44   48.55]]
